In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from GNN import TextGCN, GCN
from utils import load_corpus
from tensorflow.keras.metrics import CategoricalCrossentropy

In [115]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus("R8")

(4937, 300) (4937, 8) (2189, 300) (2189, 8) (13173, 300) (13173, 8)
15362


In [116]:
adj_dense = adj.todense().astype(np.float32)
features_dense = features.todense().astype(np.float32)

In [117]:
# features_dense[:-2189]
# features_train = features_dense[:-2189]
# features_test = features_dense[-2189:]
# y_train = y_train[:-2189]
# y_test = y_test[-2189:]

In [53]:
features_train.shape

(13173, 300)

In [110]:
len(np.where(train_mask == False)[0])

10425

In [113]:
len(np.where(train_mask == True)[0])

4937

In [84]:
adj_dense[:-5485,-5485:].shape

(9877, 5485)

In [93]:
adj_dense[:-5485,:-5485].shape

(9877, 9877)

In [106]:
a, b = 9876, 9837
adj_dense[:-5485,:-5485][a,b], adj_dense[:-5485,:-5485][b,a]

(3.241013, 3.241013)

In [103]:
np.where(adj_dense[:-5485,:-5485] != 0)

(array([   0,    0,    0, ..., 9876, 9876, 9876]),
 array([5493, 6238, 6745, ..., 9821, 9837, 9858]))

In [104]:
adj_dense[:-5485,:-5485][9876, 9821]

3.903479

In [140]:
train_mask = train_mask + val_mask

array([ True,  True,  True, ..., False, False, False])

In [154]:
np.where(np.argmax((y_train + y_val), axis=-1) == 0)

(array([   27,    76,    79, ..., 15359, 15360, 15361]),)

In [155]:
train_size, test_size

(5485, 2189)

In [118]:
model = GCN(adj_dense, layers=2, hidden_size=[64,8], dropout=0.7)

output = model(features_dense)

In [119]:
output

<tf.Tensor: shape=(15362, 8), dtype=float32, numpy=
array([[0.22746952, 0.64045167, 0.20985024, ..., 0.4818032 , 0.2408684 ,
        0.20959687],
       [0.06087749, 0.07684083, 0.02592787, ..., 0.06496865, 0.05490308,
        0.10400116],
       [0.05430752, 0.09826499, 0.17549118, ..., 0.0380405 , 0.0320874 ,
        0.02457985],
       ...,
       [0.54437405, 0.9254319 , 0.96327084, ..., 0.9321483 , 0.8277507 ,
        1.5347793 ],
       [2.2180848 , 2.0315766 , 1.9225012 , ..., 2.8852468 , 2.427651  ,
        1.9832155 ],
       [0.04187041, 0.06992984, 0.05136723, ..., 0.1640001 , 0.13316566,
        0.10362199]], dtype=float32)>

In [125]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [158]:
import scipy.sparse as sp
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose()
        values = mx.data
        shape = mx.shape
        return coords, values, shape

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return sparse_to_tuple(features)


preprocess_features(features)

(array([[ 5485,   299],
        [ 5485,   298],
        [ 5485,   297],
        ...,
        [13172,     2],
        [13172,     1],
        [13172,     0]], dtype=int32),
 array([-2.00174035,  0.03622683,  0.87804978, ...,  0.04228568,
         0.01613521, -0.1138999 ]),
 (15362, 300))

In [126]:
masked_softmax_cross_entropy(output, y_train, train_mask)

<tf.Tensor: shape=(), dtype=float32, numpy=2.1135924>

In [127]:
a = masked_softmax_cross_entropy

In [161]:
(train_mask + val_mask).sum()

5485

In [162]:
test_mask.sum()

2189

In [164]:
y_train.shape

(15362, 8)

In [128]:
a(output, y_train, train_mask)

<tf.Tensor: shape=(), dtype=float32, numpy=2.1135924>

In [132]:
masked_accuracy(output,y_test,test_mask)

<tf.Tensor: shape=(), dtype=float32, numpy=0.11146644>

In [8]:
tf.argmax(y_train, axis=1)

<tf.Tensor: shape=(15362,), dtype=int64, numpy=array([3, 2, 7, ..., 0, 0, 0])>

In [15]:
loss = CategoricalCrossentropy()
loss_val = loss(y_train, output)
loss_val

In [24]:
(tf.argmax(y_test,axis=-1) == tf.argmax(tf.nn.softmax(output,axis=0),axis=-1)).numpy().mean()

0.3472204140085926

<tf.Tensor: shape=(15362,), dtype=int64, numpy=array([0, 0, 0, ..., 3, 6, 7])>